In [2]:
"""Encode all states using best checkpoint path.

Change:
- checkpoint_path
- save_dir
- timestep in data.py
"""


import os
import sys

import torch
from torch.utils.data import DataLoader, TensorDataset

# sys.path.append('/local/scratch/ysun564/project/OfflineRL_TimeStep')

sys.path.append('F:\\time_step\\OfflineRL_FactoredActions')
from RL_mimic_sepsis.b_train_encoder.src.data import train_data_file, val_data_file, test_data_file, timestep
from RL_mimic_sepsis.b_train_encoder.src.model import AIS_GRU

In [3]:
timestep

8

In [4]:
action_space = 'NormThreshold'
# The path that stores best checkpoint path.
ckpt_txt_path = ('/local/scratch/ysun564/project/OfflineRL_TimeStep/RL_mimic_sepsis/b_train_encoder'
                f'/logs/AIS_GRU_model_as{action_space}_dt{timestep}h_grid/results/best_ckpt_path.txt')



In [7]:
# Read the txt file.
# with open(ckpt_txt_path, 'r', encoding='utf-8') as f:
#     checkpoint_path = f.read()

checkpoint_path = rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\b_train_encoder\logs\AIS_GRU_model_asNormThreshold_dt8h_best_128\epoch=471-step=47672-v1.ckpt'

In [8]:
# Load best model based on `Visualize.ipynb`
# save_dir = ('/local/scratch/ysun564/project/OfflineRL_TimeStep/RL_mimic_sepsis'
#            f'/data/data_as{action_space}_dt{timestep}h/episodes+encoded_state_128/')

save_dir = rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\data_asNormThreshold_dt8h\episodes+encoded_state_128'
os.makedirs(save_dir, exist_ok=True)
model = AIS_GRU.load_from_checkpoint(checkpoint_path=checkpoint_path, map_location=None)
model.eval()

d:\Software\anaconda3\envs\rl4h_rep_new\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.5.2, which is newer than your current Lightning version: v2.5.1.post0


AIS_GRU(
  (gen): Sequential(
    (0): Linear(in_features=63, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): GRU(128, 128, batch_first=True)
    (5): ExtractRNNOutput()
  )
  (pred): Sequential(
    (0): Linear(in_features=153, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)

In [9]:
train_data_file

'F:\\time_step\\OfflineRL_FactoredActions\\RL_mimic_sepsis\\data/data_asNormThreshold_dt8h\\episodes/train_set_asNormThreshold_dt8h.pt'

In [10]:
train_data = torch.load(train_data_file)
val_data = torch.load(val_data_file)
test_data = torch.load(test_data_file)

In [11]:
# Check consistency of sample size before proceeding
for data in [train_data, val_data, test_data]:
    num_episodes = len(data['index'])
    for k, v in data.items():
        assert len(v) == num_episodes

In [12]:
train_dataset = TensorDataset(
    train_data['demographics'], 
    train_data['observations'], 
    train_data['actionvecs'], 
    train_data['lengths'], 
)
val_dataset = TensorDataset(
    val_data['demographics'], 
    val_data['observations'], 
    val_data['actionvecs'], 
    val_data['lengths'], 
)
test_dataset = TensorDataset(
    test_data['demographics'], 
    test_data['observations'], 
    test_data['actionvecs'], 
    test_data['lengths'], 
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [13]:
device = next(model.parameters()).device
# Encode all states.
all_encoded_states = []
for dataloader in [train_loader, val_loader, test_loader]:
    with torch.no_grad():
        encoded_states = []
        for dem, obs, act, lng in dataloader:
            dem = dem.to(device)
            obs = obs.to(device)
            act = act.to(device)
            x = torch.cat([obs, dem, act], dim=-1)
            z = model(x)
            mask = (obs == 0).all(dim=2)
            z[mask] = 0
            encoded_states.append(z)

    encoded_states = torch.cat(encoded_states, axis=0)
    all_encoded_states.append(encoded_states)

In [14]:
train_states, val_states, test_states = all_encoded_states

In [15]:
train_data['statevecs'] = train_states
val_data['statevecs'] = val_states
test_data['statevecs'] = test_states

In [16]:
# Check consistency of sample size of outputs
for data in [train_data, val_data, test_data]:
    num_episodes = len(data['index'])
    for k, v in data.items():
        assert len(v) == num_episodes

In [17]:
torch.save(train_data, os.path.join(save_dir, 'train_data.pt'))
torch.save(val_data, os.path.join(save_dir, 'val_data.pt'))
torch.save(test_data, os.path.join(save_dir, 'test_data.pt'))

In [18]:
# Check consistency of episode length and feature vector
for i in range(100):
    lng = train_data['lengths'][i]
    assert (train_data['observations'][i][lng:] == 0).all()
    assert (train_data['statevecs'][i][lng:] == 0).all()